## Алгоритмы анализа данных.
### Урок 3. Логистическая регрессия. Log Loss.

In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [59]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)

In [60]:
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [61]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [62]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [63]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    err = np.sum(err)
    return err

In [64]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [65]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

#### 1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [66]:
def calc_logloss(y, y_pred):
    predicted = y_pred.copy()
    for i in range(len(predicted)):
        if predicted[i] == 0.0:
            predicted[i] = 0.01
        elif predicted[i] == 1.0:
            predicted[i] = 0.99
      
    err = - np.mean(y * np.log(predicted) + (1.0 - y) * np.log(1.0 - predicted))
    err = np.sum(err)
    return err

#### 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [67]:
def eval_model_(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    return W, err, alpha, iterations

In [68]:
best_params = None
err_min = 1000
i = 0
for iterations in [100, 250, 500, 750, 1000, 2000, 3000, 4000, 5000]:
    for alpha in [1e-3, 1e-8, 1e-2, 0.005, 1e-5, 1e-4]:
        params = W, err, alpha, iterations = eval_model_(X_st, y, iterations=iterations, alpha=alpha)
        if err < err_min:
            err_min = err
            best_params = params
print(f'Best parameters: Iteration {params[-1]}, alpha {params[-2]}')

Best parameters: Iteration 5000, alpha 0.0001


#### 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [69]:
W = eval_model(X_st, y, iterations=3000, alpha=1e-4)

300 [ 0.48550145 -0.18121987  0.64677288  1.50625373] 1.1037319870460949
600 [ 0.47464147 -0.22295121  0.64567425  1.48992272] 1.0328037733069197
900 [ 0.46415304 -0.26333611  0.64441017  1.47408065] 0.9663350259228656
1200 [ 0.45405295 -0.30224157  0.64301079  1.45877701] 0.9045879597845504
1500 [ 0.44435675 -0.33951515  0.64152026  1.44406912] 0.8478360281905125
1800 [ 0.43507949 -0.37497925  0.63999658  1.43002306] 0.7963644084220479
2100 [ 0.42623565 -0.40843447  0.63850898  1.41671156] 0.7504398545631022
2400 [ 0.41783764 -0.43967723  0.63713209  1.4042075 ] 0.7102452170811083
2700 [ 0.40989301 -0.46853111  0.63593713  1.39257337] 0.6757964688435114
3000 [ 0.40240136 -0.49488241  0.63498247  1.38184975] 0.6468813805442177


In [70]:
def calc_pred_proba(X, W):
    z = np.dot(W, X)
    y_pred_proba = np.floor(sigmoid(z))
    return y_pred_proba

In [71]:
calc_pred_proba(X, W)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

#### 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [72]:
def calc_pred(W, X):
    m = X.shape[1]
    y_pred = np.zeros((1, m), dtype = np.int64)
    z = W.T @ X
    y_pred_proba = sigmoid(z)
    
    for i in range(len(y_pred_proba)):
        if (y_pred_proba[i] > 0.5): 
            y_pred[:, i] = 1
        elif (y_pred_proba[i] <= 0.5):
            y_pred[:, i] = 0
    return y_pred

In [73]:
y_pred = calc_pred(W, X)
y_pred

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int64)

#### 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

**Accuracy**

In [74]:
def accuracy_(y_pred, y):
    return len(y_pred[y == y_pred]) / len(y_pred)

In [75]:
accuracy_(y_pred, y)

5.0

**Correlation Matrix**

In [76]:
def correlation_matrix(y_pred, y):
    TP = len(y_pred[(y == y_pred) & (y_pred == 1)])
    FN = len(y_pred[(y != y_pred) & (y_pred == 0)])
    FP = len(y_pred[(y != y_pred) & (y_pred == 1)])
    TN = len(y_pred[(y == y_pred) & (y_pred == 0)])
    corr_matrix = pd.DataFrame.from_dict(
        {'y_pred': ['a(x) = +1', 'a(x) = -1'], 
         'y = +1': [TP, FN], 
         'y = -1': [FP, TN]}).set_index('y_pred')
    return corr_matrix

In [77]:
correlation_matrix(y_pred, y)

,y = +1,y = -1
y_pred,,
a(x) = +1,5,5
a(x) = -1,0,0


**Precision**

In [78]:
def precision_(y_pred, y):
    return len(y_pred[(y == y_pred) & (y_pred == 1)]) / (
        len(y_pred[(y == y_pred) & (y_pred == 1)]) + len(y_pred[(y != y_pred) & (y_pred == 1)]))

In [79]:
precision_(y_pred, y)

0.5

**Recall**

In [80]:
def recall_(y_pred, y):
    return len(y_pred[(y == y_pred) & (y_pred == 1)]) / (
        len(y_pred[(y == y_pred) & (y_pred == 1)]) + len(y_pred[(y != y_pred) & (y_pred == 0)]))

In [81]:
recall_(y_pred, y)

1.0

**F1-score**

In [82]:
def f1_(y_pred, y):
    return 2 * precision_(y_pred, y) * recall_(y_pred, y) / (precision_(y_pred, y) + recall_(y_pred, y))

In [83]:
f1_(y_pred, y)

0.6666666666666666

#### 6. Могла ли модель переобучиться? Почему?

Да, переобучение модели возможно. Из-за недостаточности данных.